In [2]:
#install vggface
!pip install git+https://github.com/rcmalli/keras-vggface.git
# Release Version
!pip install keras_vggface

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-gpf41c9s
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-gpf41c9s


In [3]:
#install keras
!pip install keras
!pip install --upgrade setuptools
!pip install cmake
!pip install keras-models
!pip install keras-applications
!pip install keras-preprocessing

     |████████████████████████████████| 953 kB 12.5 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 50 kB 4.9 MB/s 


In [4]:
#mounting
from google.colab import drive
drive.mount('/content/drive') 

%cd drive/MyDrive/Dataset/SKRIPSI

MessageError: ignored

In [6]:
#library
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.layers import Conv2D, Dense, BatchNormalization, Activation, Dropout, MaxPooling2D, Flatten
# from keras.optimizers import RMSprop, SGD
from keras import regularizers
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
import datetime
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model


In [7]:
#GATHER DATA
train_dir = 'dataset/balance_overtr7k/'
test_dir = 'dataset/test/'

row, col = 48, 48
classes = 7

def count_exp(path, set_):
    dict_ = {}
    for expression in os.listdir(path):
        dir_ = path + expression
        dict_[expression] = len(os.listdir(dir_))
    df = pd.DataFrame(dict_, index=[set_])
    return df

#MENAMPILKAN JUMLAH DATASET
train_count = count_exp(train_dir, 'train')
test_count = count_exp(test_dir, 'test')
print(train_count)
print(test_count)

FileNotFoundError: ignored

In [ ]:
#MENAMPILKAN CONTOH DATA
plt.figure(figsize=(14,22))
i = 1
for expression in os.listdir(train_dir):
    img = load_img((train_dir + expression +'/'+ os.listdir(train_dir + expression)[5]))
    plt.subplot(1,7,i)
    plt.imshow(img)
    plt.title(expression)
    plt.axis('off')
    i += 1
plt.show()

In [ ]:
#PREPARING DATA DENGAN RESIZE KE 224 UNTUK VGGFACE
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.3,
                                   horizontal_flip=True)

training_set = train_datagen.flow_from_directory(train_dir,
                                                batch_size=64,
                                                target_size=(224,224),
                                                shuffle=True,
                                                color_mode='rgb',
                                                class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(test_dir,
                                                batch_size=64,
                                                target_size=(224,224),
                                                shuffle=True,
                                                color_mode='rgb',
                                                class_mode='categorical')

In [ ]:
training_set.class_indices

In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip install keras_applications --no-deps
filename = "/usr/local/lib/python3.7/dist-packages/keras_vggface/models.py"
text = open(filename).read()
open(filename, "w+").write(text.replace('keras.engine.topology', 'tensorflow.keras.utils'))
import tensorflow as tf

from keras_vggface.vggface import VGGFace

vggface = VGGFace(model='resnet50') # or VGGFace() as default

In [ ]:
#IMPORT VGGFACE
# from keras.utils.layer_utils import get_source_inputs
from keras_vggface.vggface import VGGFace

#VGGFACE SENET50
base_model = VGGFace(model='senet50')

In [ ]:
#PREPARING LAYER VGGFACE
for layer in base_model.layers[:-4]:
    layer.trainable=False

In [ ]:
#LAYER DIAKHIRI DENGAN DENSE 7
from keras.models import Sequential

model=Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(7,activation='softmax'))

#COMPILE MODEL
model.compile(optimizer='Adam', loss='categorical_crossentropy')

In [ ]:
#TUNING
from keras.callbacks import LearningRateScheduler
import math
from math import pi
from math import cos
chk_path = 'ferNet.h5'
log_dir = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#INISIALISASI ARRAY LEARNINGRATE
aa = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]

#MENGUBAH ARRAY LEARNINGRATE MENJADI INCREMENTAL COSINE ANNEALING
def ica(epoch):
    if(epoch % 10 == 0):
      lrate = aa[9]
      return lrate
    else:
      if(epoch <=10):
        cos_inner = (pi * (epoch % 10)) / (10)
        lrate = 0.014/2 * (cos(cos_inner) + 1)
        return lrate
      elif(i>10 and i<20):
        cos_inner = (pi * (epoch % 10)) / (10)
        lrate = 0.015/2 * (cos(cos_inner) + 1)
        return lrate
      elif(epoch>20 and i<30):
        cos_inner = (pi * (epoch % 10)) / (10)
        lrate = 0.016/2 * (cos(cos_inner) + 1)
        return lrate
      elif(epoch>30 and epoch<40):
        cos_inner = (pi * (epoch % 10)) / (10)
        lrate = 0.016/2 * (cos(cos_inner) + 1)
        return lrate
      elif(i>40):
        cos_inner = (pi * (epoch % 10)) / (10)
        lrate = 0.017/2 * (cos(cos_inner) + 1)
        return lrate

lrate = LearningRateScheduler(ica)

checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             verbose=1,
                             mode='min',
                             moniter='val_loss')

csv_logger = CSVLogger('training.log')

callbacks = [checkpoint, csv_logger]

In [ ]:
#TRAINING

steps_per_epoch = training_set.n // training_set.batch_size
validation_steps = test_set.n // test_set.batch_size

hist = model.fit(x=training_set,
                 validation_data=test_set,
                 epochs=50,
                 callbacks=callbacks,
                 steps_per_epoch=steps_per_epoch,
                 validation_steps=validation_steps)

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(1,2,2)
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(1,2,1)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#evaluasi train, validation menggunakan test

train_loss, train_accu = model.evaluate(training_set)
test_loss, test_accu = model.evaluate(test_set)
print("final train accuracy = {:.2f} , validation accuracy = {:.2f}".format(train_accu*100, test_accu*100))